In [1]:
import numpy as np
import pandas as pd
from datetime import datetime

In [2]:
covid_deaths = pd.read_csv('data/Deaths.csv')

In [3]:
today_date = datetime.now().strftime('%m/%d/%y').lstrip("0").replace(" 0", " ")
today_date

'3/24/20'

In [4]:
covid_deaths.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20
0,NaN,Thailand,15.0000,101.0000,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1.0
1,NaN,Japan,36.0000,138.0000,0,0,0,0,0,0,...,22,22,27,29,29,29,33,35,40,40.0
2,NaN,Singapore,1.2833,103.8333,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,2,2.0
3,NaN,Nepal,28.1667,84.2500,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
4,NaN,Malaysia,2.5000,112.5000,0,0,0,0,0,0,...,0,0,0,2,2,2,3,4,10,10.0


In [5]:
country_locations = covid_deaths[['Country/Region', 'Lat', 'Long']]
country_locations

,Country/Region,Lat,Long
0,Thailand,15.0000,101.0000
1,Japan,36.0000,138.0000
2,Singapore,1.2833,103.8333
3,Nepal,28.1667,84.2500
4,Malaysia,2.5000,112.5000
...,...,...,...
496,Jersey,49.1900,-2.1100
497,Puerto Rico,18.2000,-66.5000
498,Republic of the Congo,-1.4400,15.5560
499,The Bahamas,24.2500,-76.0000


In [6]:
current_totals = covid_deaths[['Country/Region', today_date]]
current_totals

KeyError: "['3/24/20'] not in index"

In [ ]:
countries_set = set(current_totals['Country/Region'])
country_totals = {'Country': list(countries_set), 'Total': []} 
for country in countries_set:
    df = current_totals[current_totals['Country/Region'] == country]
    country_totals['Total'].append(df[today_date].sum())
country_totals = pd.DataFrame(country_totals)
country_totals = country_totals.set_index('Country')
country_totals

In [ ]:
populations = pd.read_csv('data/TotalPopulationBySex.csv')
populations = populations[populations["Time"] == 2020]
populations.head()

In [ ]:
countries_a = list(country_totals.index)
countries_b = populations['Location'].unique()

shared = [country_name for country_name in countries_a if country_name in countries_b]
not_shared_a = [country_name for country_name in countries_a if country_name not in countries_b]
not_shared_b = [country_name for country_name in countries_b if country_name not in countries_a]

In [ ]:
not_shared_a.sort()
not_shared_a

In [ ]:
not_shared_b.sort()
not_shared_b

In [ ]:
spelling_changes = {
    'Bahamas': 'Bahamas, The',
    'Bolivia (Plurinational State of)': 'Bolivia',
    'Brunei Darussalam': 'Brunei',
    'Côte d\'Ivoire': 'Cote d\'Ivoire',
    'Gambia': 'Gambia, The',
    'Iran (Islamic Republic of)': 'Iran',
    'Republic of Korea': 'Korea, South',
    'Russian Federation': 'Russia',
    'Syrian Arab Republic': 'Syria',
    'China, Taiwan Province of China': 'Taiwan*',
    'United Republic of Tanzania': 'Tanzania',
    'United States of America': 'US',
    'Venezuela (Bolivarian Republic of)': 'Venezuela',
    'Viet Nam': 'Vietnam'
}

In [ ]:
current_populations = {'Country': [], 'Population': []}
for country_name in populations['Location'].unique():
    if country_name in spelling_changes.keys():
        key_name = spelling_changes[country_name]
    elif country_name not in shared:
        continue
    else:
        key_name = country_name
    current_populations['Country'].append(key_name)
    current_populations['Population'].append(populations[populations['Location'] == country_name]['PopTotal'].iloc[0]) 
current_populations = pd.DataFrame(current_populations)
current_populations = current_populations.set_index('Country')
current_populations

In [ ]:
country_totals = country_totals.join(current_populations)
country_totals

In [ ]:
country_totals['Death rate'] = country_totals['Total'] / country_totals['Population']
country_totals.sort_values(by=['Death rate'], ascending=False).head(20)